# Configuration Access

<b>This notebook is not supposed to be used on its own.<b>

In [ ]:
def get_access_store_ui(root_dir: str = '.'):
    
    from pathlib import Path
    from enum import Enum
    import ipywidgets as widgets
    from exasol.secret_store import Secrets as Secrets_
    
    # A fix for the secret store to make its interface more in line with the dict.
    class Secrets(Secrets_):
        
        def get(self, key: str, default_value: str=None) -> str:
            value = super().get(key)
            return default_value if value is None else value

        def __getattr__(self, key):
            value = self.get(key)
            if value is None:
                raise AttributeError(f'{key} value is not defined')
            return value

    # Try to find the file name in the shared store.
    # Create a global variable only temporarily.
    %store -r
    if 'sb_store_file' in globals():
        global sb_store_file
        sb_store_file_ = sb_store_file
        del sb_store_file
    else:
        sb_store_file_ = 'dss_config.sqlite'

    WKey = Enum('WKey', ['file', 'password', 'ok'])
    style = {'description_width': '140px'}
    inputs = {
        WKey.file:     widgets.Text(description='Config. File Name', value=sb_store_file_, style=style),
        WKey.password: widgets.Password(description='Config. Password', style=style),
        WKey.ok:       widgets.Button(description='OK')
    }

    def open_or_create_config_store(btn):
        global sb_config, sb_store_file
        sb_store_file = inputs[WKey.file].value
        sb_config = Secrets(Path(root_dir) / sb_store_file, inputs[WKey.password].value)
        # Save the file in the shared store.
        %store sb_store_file
        del sb_store_file
    
    inputs[WKey.ok].on_click(open_or_create_config_store)

    ui = widgets.VBox(list(inputs.values()))
    return ui